URL we got things from was 

https://brickset.com/exportscripts/inventory/

+SetNumber+'-1'

SetNumber is the set number, USE

PartID is the DesignID + color

Quantity is the count, USE

Colour is the color

DesignID is the block shape, USE

PartName is the name of the part, USE

SetCount is the number of sets this part is in

In [1]:
import sys, os

import matplotlib.pyplot as plt

plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = (14,10)

%matplotlib inline

import pandas as pd

In [2]:
useable_blocks = [
                    'plate_1x1' ,
                    'plate_1x2' ,
                    'plate_1x3' ,
                    'plate_1x4' ,
                    'plate_1x6' ,
                    'plate_1x8' ,
                    'plate_1x10',
                    'plate_2x2' ,
                    'plate_2x3' ,
                    'plate_2x4' ,
                    'plate_2x6' ,
                    'plate_2x8' ,
                    'plate_2x12',
                    'plate_4x4' ,
                    'plate_4x6' ,
                    'plate_4x8' ,
                    'plate_6x6' ,
                    'plate_6x8' ,
                    'plate_6x10',
                    'plate_8x8' ,
                    'brick_1x1' ,
                    'brick_1x2' ,
                    'brick_1x3' ,
                    'brick_1x4' ,
                    'brick_1x6' ,
                    'brick_1x8' ,
                    'brick_2x2' ,
                    'brick_2x3' ,
                    'brick_2x4' ,
                    'brick_2x6' ,
                 ]

set_dir = 'set_csvs/'

file_list = os.listdir( set_dir )

In [3]:
out_frame = pd.DataFrame( 
                            columns=['set_id','set_name','set_url','n_parts',]+useable_blocks,
                        )

for i in range( 0, len(file_list) ):
    csv = file_list[i]

    # The csv file from the website
    inp_set_df = pd.read_csv( set_dir + csv )


    # The useful columns
    use_set_df = inp_set_df[['SetNumber','Quantity','DesignID','PartName']].copy()


    # Get list of all the bricks in the set, and all the plates
    brick_list  = [ block for block in use_set_df['PartName'].unique() if ( ('Brick' in block) and ( len(block) < 11 ) ) ]
    plate_list  = [ block for block in use_set_df['PartName'].unique() if ( ('Plate' in block) and ( len(block) < 11 ) ) ]


    # Get the sum of the plates and bricks
    part_sum = use_set_df.groupby('PartName')['Quantity'].sum()
    part_sum = part_sum[ brick_list + plate_list ]


    # Change from Brick 1X2 to brick_1x2
    better_index    = [ index.replace(' ','_').lower() for index in part_sum.index ]
    part_sum.index  = better_index


    # Make a list of count of all the parts we track
    part_list_count = [ part_sum[ind] if ( ind in part_sum.index ) else 0 for ind in useable_blocks ]


    # Get the total number of parts in the set
    total_parts = use_set_df['Quantity'].sum()

    out_frame.loc[i] = [
                        use_set_df['SetNumber'].values[0],
                        use_set_df['SetNumber'].values[0]+'_NAME',
                        use_set_df['SetNumber'].values[0]+'_URL',
                        total_parts
                       ]+part_list_count

ValueError: cannot set a row with mismatched columns

In [5]:
out_frame.head()

,set_id,n_parts,plate_1x1,plate_1x2,plate_1x3,plate_1x4,plate_1x6,plate_1x8,plate_1x10,plate_2x2,...,brick_1x1,brick_1x2,brick_1x3,brick_1x4,brick_1x6,brick_1x8,brick_2x2,brick_2x3,brick_2x4,brick_2x6
0,7992-1,213,0,5,8,4,4,5,0,3,...,2,2,0,2,0,2,0,0,1,2
1,1088-1,24,0,0,2,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,60024-1,241,0,12,2,5,1,0,0,1,...,2,2,0,0,0,0,0,0,0,0
3,6454-1,103,0,6,0,0,2,0,0,1,...,2,3,0,0,0,0,0,0,0,2
4,7623-1,553,15,26,0,5,0,4,5,3,...,0,4,3,0,3,2,2,2,4,0


In [6]:
out_frame.to_csv('lego_set_parts.csv')